# Homework 3
In this assignment, we will be building a Naïve Bayes classifier and a SVM model for the productivity satisfaction of [the given dataset](https://archive.ics.uci.edu/ml/datasets/Productivity+Prediction+of+Garment+Employees), the productivity of garment employees.

## Background
The Garment Industry is one of the key examples of the industrial globalization of this modern era. It is a highly labour-intensive industry with lots of manual processes. Satisfying the huge global demand for garment products is mostly dependent on the production and delivery performance of the employees in the garment manufacturing companies. So, it is highly desirable among the decision makers in the garments industry to track, analyse and predict the productivity performance of the working teams in their factories.

## Dataset Attribute Information

1. **date**: Date in MM-DD-YYYY
2. **day**: Day of the Week
3. **quarter** : A portion of the month. A month was divided into four quarters
4. **department** : Associated department with the instance
5. **team_no** : Associated team number with the instance
6. **no_of_workers** : Number of workers in each team
7. **no_of_style_change** : Number of changes in the style of a particular product
8. **targeted_productivity** : Targeted productivity set by the Authority for each team for each day.
9. **smv** : Standard Minute Value, it is the allocated time for a task
10. **wip** : Work in progress. Includes the number of unfinished items for products
11. **over_time** : Represents the amount of overtime by each team in minutes
12. **incentive** : Represents the amount of financial incentive (in BDT) that enables or motivates a particular course of action.
13. **idle_time** : The amount of time when the production was interrupted due to several reasons
14. **idle_men** : The number of workers who were idle due to production interruption
15. **actual_productivity** : The actual % of productivity that was delivered by the workers. It ranges from 0-1.

### Libraries that can be used: numpy, scipy, pandas, scikit-learn, cvxpy, imbalanced-learn
Any libraries used in the discussion materials are also allowed.

#### Other Notes

 - Don't worry about not being able to achieve high accuracy, it is neither the goal nor the grading standard of this assignment. <br >
 - If not specified, you are not required to do hyperparameter tuning, but feel free to do so if you'd like.

#### Trouble Shooting
In case you have trouble installing and using imbalanced-learn(imblearn) <br >
Run the below code cell, then go to the selection bar at top: Kernel > Restart. <br >
Then try `import imblearn` to see if things work.

In [73]:
import platform
display(platform.system())
import os
file_download_link = 'https://www.dropbox.com/scl/fi/j1dxtqjerbdbl81e05nvp/hw4data.zip?rlkey=8mkxz4j8lngziok6on782a8u4&dl=0'
if os.name == 'nt':
    print('Please download your dataset here:', file_download_link)
else:
    # We need to first download the data here:
    !wget -O data.zip "$file_download_link" -o /dev/null
    !unzip data.zip > /dev/null

'Linux'

replace garments_worker_productivity.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [74]:
# If your data is on google drive then uncomment the code below to access
# your google drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [75]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install imbalanced-learn delayed

# Exercises

## Exercise 1 - General Data Preprocessing (20 points)

Our dataset needs cleaning before building any models. Some of the cleaning tasks are common in general, but depends on what kind of models we are building, sometimes we have to do additional processing. These additional tasks will be mentioned in each of the remaining two exercises later.

Note that **we will be using this processed data from exercise 1 in each of the remaining two exercises**.

For convenience, here are the attributes that we would treat them as **categorical attributes**: `day`, `quarter`, `department`, and `team`.

 - Drop the column `date`.
 - For each of the categorical attributes, **print out** all the unique elements.
 - For each of the categorical attributes, remap the duplicated items, if you find there are typos or spaces among the duplicated items.
     - For example, "a" and "a " should be the same, so we need to update "a " to be "a".
     - Another example, "apple" and "appel" should be the same, so you should update "appel" to be "apple".
     

 - Create another column named `satisfied` that records the productivity performance. The behavior defined as follows. **This is the dependent variable we'd like to classify in this assignment.**
     - Return True or 1 if `actual_productivity` is equal to or greater than `targeted_productivity`. Otherwise, return False or 0, which means the team fails to meet the expected performance.
 - Drop the columns `actual_productivity` and `targeted_productivity`.


 - Find and **print out** which columns/attributes that have empty vaules, e.g., NA, NaN, null, None.
 - Fill the empty values with 0.


In [76]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report,confusion_matrix, ConfusionMatrixDisplay
from sklearn.naive_bayes import CategoricalNB, GaussianNB
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC

# If put the data(.csv) under the same folder, you could use
df = pd.read_csv('./garments_worker_productivity.csv')
df = df.drop('date', axis=1)

display(df.day.unique())
display(df.quarter.unique())
display(df.department.unique())
display(df.team.unique())

df['department'] = df['department'].replace(['finishing '], 'finishing')

satisfied = []
for i in df.index:
    if df['actual_productivity'][i] >= df['targeted_productivity'][i]:
      satisfied.append(1)
    else:
      satisfied.append(0)
df['satisfied'] = satisfied
df = df.drop('actual_productivity', axis=1)
df = df.drop('targeted_productivity', axis=1)

col_empty_vals = []

for col in df.columns:

    if df[col].isnull().any():
        col_empty_vals.append(col)

# Print the columns with empty values
for column in col_empty_vals:
    print(column)
df.fillna(0, inplace = True)
df

array(['Thursday', 'Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday'],
      dtype=object)

array(['Quarter1', 'Quarter2', 'Quarter3', 'Quarter4', 'Quarter5'],
      dtype=object)

array(['sweing', 'finishing ', 'finishing'], dtype=object)

array([ 8,  1, 11, 12,  6,  7,  2,  3,  9, 10,  5,  4])

wip


,quarter,department,day,team,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,satisfied
0,Quarter1,sweing,Thursday,8,26.16,1108.0,7080,98,0.0,0,0,59.0,1
1,Quarter1,finishing,Thursday,1,3.94,0.0,960,0,0.0,0,0,8.0,1
2,Quarter1,sweing,Thursday,11,11.41,968.0,3660,50,0.0,0,0,30.5,1
3,Quarter1,sweing,Thursday,12,11.41,968.0,3660,50,0.0,0,0,30.5,1
4,Quarter1,sweing,Thursday,6,25.90,1170.0,1920,50,0.0,0,0,56.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,Quarter2,finishing,Wednesday,10,2.90,0.0,960,0,0.0,0,0,8.0,0
1193,Quarter2,finishing,Wednesday,8,3.90,0.0,960,0,0.0,0,0,8.0,0
1194,Quarter2,finishing,Wednesday,7,3.90,0.0,960,0,0.0,0,0,8.0,0
1195,Quarter2,finishing,Wednesday,9,2.90,0.0,1800,0,0.0,0,0,15.0,0


## Exercise 2 - Naïve Bayes Classifier (40 points in total)

### Exercise 2.1 - Additional Data Preprocessing (10 points)

To build a Naïve Bayes Classifier, we need to further encode our categorical variables.

 - For each of the **categorical attribtues**, encode the set of categories to be **0 ~ (n_classes - 1)**.
     - For example, \["paris", "paris", "tokyo", "amsterdam"\] should be encoded as \[1, 1, 2, 0\].
     - Note that the order does not really matter, i.e., \[0, 0, 1, 2\] also works. But you have to start with 0 in your encodings.
     - You can find information about this encoding in the discussion materials.


 - Split the data into training and testing set with the ratio of 80:20.

In [77]:
# Remember to continue the task with your processed data from Exercise 1
encode = {
    "quarter": {"Quarter1": 0, "Quarter2": 1, "Quarter3": 2, "Quarter4": 3, "Quarter5": 4},
    "department": {"finishing": 0, "sweing": 1},
    "day": {"Monday": 0, "Tuesday": 1, "Wednesday": 2, "Thursday": 3, "Saturday": 5, "Sunday": 6},
    "team": {8: 0, 1: 1, 11: 2, 12: 3, 6: 5, 7: 6, 2: 7, 3: 9, 9: 10, 10: 11, 5: 12, 4: 13}
}

df = df.replace(encode)
df

,quarter,department,day,team,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,satisfied
0,0,1,3,0,26.16,1108.0,7080,98,0.0,0,0,59.0,1
1,0,0,3,1,3.94,0.0,960,0,0.0,0,0,8.0,1
2,0,1,3,2,11.41,968.0,3660,50,0.0,0,0,30.5,1
3,0,1,3,3,11.41,968.0,3660,50,0.0,0,0,30.5,1
4,0,1,3,5,25.90,1170.0,1920,50,0.0,0,0,56.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,1,0,2,11,2.90,0.0,960,0,0.0,0,0,8.0,0
1193,1,0,2,0,3.90,0.0,960,0,0.0,0,0,8.0,0
1194,1,0,2,6,3.90,0.0,960,0,0.0,0,0,8.0,0
1195,1,0,2,10,2.90,0.0,1800,0,0.0,0,0,15.0,0


### Exercise 2.2 - Naïve Bayes Classifier for Categorical Attributes (15 points)

Use the categorical attributes **only**, please build a Categorical Naïve Bayes classifier that predicts the column `satisfied`. <br >
Report the **testing result** using `classification_report`.

In [78]:
# Remember to do this task with your processed data from Exercise 2.1
data_to_drop = ['satisfied','smv','wip', 'over_time', 'incentive', 'idle_time', 'idle_men', 'no_of_style_change','no_of_workers']
X_train, X_test, y_train, y_test = train_test_split(df.drop(data_to_drop, axis=1), df.satisfied, test_size=0.2, random_state=13)
NB = CategoricalNB()
NB.fit(X_train,y_train)
yhat_test = NB.predict(X_test)
print(classification_report(y_test, yhat_test))

              precision    recall  f1-score   support

           0       0.59      0.20      0.30        65
           1       0.76      0.95      0.84       175

    accuracy                           0.75       240
   macro avg       0.68      0.57      0.57       240
weighted avg       0.72      0.75      0.70       240



### Exercise 2.3 - Naïve Bayes Classifier for Numerical Attributes (15 points)

Use the numerical attributes **only**, please build a Gaussian Naïve Bayes classifier that predicts the column `satisfied`. <br >
Report the **testing result** using `classification_report`.

**Remember to scale your data. The scaling method is up to you.**

In [79]:
# Remember to do this task with your processed data from Exercise 2.1
X_train, X_test, y_train, y_test = train_test_split(df.drop(['satisfied','quarter','department', 'day', 'team'], axis=1), df.satisfied, test_size=0.2, random_state=13)

scaler = preprocessing.MinMaxScaler()

scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
NB = GaussianNB()
NB.fit(X_train,y_train)
yhat_test = NB.predict(X_test)
print(classification_report(y_test, yhat_test))

              precision    recall  f1-score   support

           0       0.38      0.05      0.08        65
           1       0.73      0.97      0.84       175

    accuracy                           0.72       240
   macro avg       0.55      0.51      0.46       240
weighted avg       0.64      0.72      0.63       240



## Exercies 3 - SVM Classifier (35 points in total)

### Exercise 3.1 - Additional Data Preprocessing (10 points)

To build a SVM Classifier, we need a different encoding for our categorical variables.

 - For each of the **categorical attribtues**, encode them with **one-hot encoding**.
     - You can find information about this encoding in the discussion materials.


 - Split the data into training and testing set with the ratio of 80:20.


In [80]:
# Remember to continue the task with your processed data from Exercise 1
columns_to_drop = ['satisfied', 'smv', 'wip', 'over_time', 'incentive', 'idle_time', 'idle_men', 'no_of_style_change', 'no_of_workers']
df_encoded = df.drop(columns_to_drop, axis=1)
target = df['satisfied']
categorical_attributes = ['quarter', 'department', 'day', 'team']
encoder = OneHotEncoder(sparse_output=False)
encoded_attributes = encoder.fit_transform(df_encoded[categorical_attributes])

X_train, X_test, y_train, y_test = train_test_split(df.drop(['satisfied'], axis=1), df.satisfied, test_size=0.2, random_state=13)
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### Exercise 3.2 - SVM with Different Kernels (15 points)

Using all the attributes we have, please build a SVM that predicts the column `satisfied`. <br >
Specifically, please
 - Build one SVM with **linear kernel**.
 - Build another SVM but with **rbf kernel**.
 - Report the **testing results** of **both models** using `classification report`.

The kernel is the only setting requirement. <br >
Other hyperparameter tuning is not required. But make sure they are the same in these two SVMs if you'd like to tune the model. In other words, the only difference between the two SVMs should be the kernel setting.

**Remember to scale your data. The scaling method is up to you.**

In [81]:
# Remember to do this task with your processed data from Exercise 3.1
svm_linear = SVC(kernel = 'linear')
svm_linear.fit(X_train,y_train)
yhat_linear = svm_linear.predict(X_test)

print(classification_report(y_test, yhat_linear))

svm_rbf = SVC(kernel = 'rbf')
svm_rbf.fit(X_train,y_train)
yhatrbf = svm_rbf.predict(X_test)

print(classification_report(y_test, yhatrbf))

              precision    recall  f1-score   support

           0       0.50      0.02      0.03        65
           1       0.73      0.99      0.84       175

    accuracy                           0.73       240
   macro avg       0.62      0.50      0.44       240
weighted avg       0.67      0.73      0.62       240

              precision    recall  f1-score   support

           0       0.50      0.02      0.03        65
           1       0.73      0.99      0.84       175

    accuracy                           0.73       240
   macro avg       0.62      0.50      0.44       240
weighted avg       0.67      0.73      0.62       240



### Exercise 3.3 - SVM with Over-sampling (10 points)
 - For the column `satisfied` in our **training set**, please **print out** the frequency of each class.
 - Oversample the **training data**.
 - For the column `satisfied` in the oversampled data, **print out** the frequency of each class  again.
 - Re-build the 2 SVMs with the same setting you have in Exercise 3.2, but **use oversampled training data** instead.
     - Do not forget to scale the data first. As always, the scaling method is up to you.
 - Report the **testing result** with `classification_report`.

You can use ANY methods listed on [here](https://imbalanced-learn.org/stable/references/over_sampling.html#) such as RandomOverSampler or SMOTE. <br >
You are definitely welcomed to build your own oversampler. <br >

Note that you do not have to over-sample your testing data.

In [82]:
# Remember to do this task with your processed data from Exercise 3.1
print(y_train.value_counts())

from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state = 13)
smote_X_train, smote_y_train = smote.fit_resample(X_train, y_train)
print("oversampled data\n", smote_y_train.value_counts())
svm_linear = SVC(kernel = 'linear')
svm_linear.fit(smote_X_train,smote_y_train)
yhat_linear = svm_linear.predict(X_test)

print(classification_report(y_test, yhat_linear))

svm_poly = SVC(kernel = 'rbf')
svm_poly.fit(smote_X_train,smote_y_train)
yhatpoly = svm_poly.predict(X_test)

print(classification_report(y_test, yhatpoly))

1    700
0    257
Name: satisfied, dtype: int64
oversampled data
 1    700
0    700
Name: satisfied, dtype: int64
              precision    recall  f1-score   support

           0       0.43      0.69      0.53        65
           1       0.85      0.66      0.74       175

    accuracy                           0.67       240
   macro avg       0.64      0.67      0.64       240
weighted avg       0.74      0.67      0.68       240

              precision    recall  f1-score   support

           0       0.41      0.69      0.51        65
           1       0.85      0.63      0.72       175

    accuracy                           0.65       240
   macro avg       0.63      0.66      0.62       240
weighted avg       0.73      0.65      0.67       240



## 4) Collaborative Statement (5 points)
#### You must fill this out even if you worked alone to get credit.

It is mandatory to include a Statement of Collaboration in each submission, that follows the guidelines below.
Include the names of everyone involved in the discussions (especially in-person ones), and what was discussed.
All students are required to follow the academic honesty guidelines posted on the course website. For
programming assignments in particular, I encourage students to organize (perhaps using Piazza) to discuss the
task descriptions, requirements, possible bugs in the support code, and the relevant technical content before they
start working on it. However, you should not discuss the specific solutions, and as a guiding principle, you are
not allowed to take anything written or drawn away from these discussions (no photographs of the blackboard,
written notes, referring to Piazza, etc.). Especially after you have started working on the assignment, try to restrict
the discussion to Piazza as much as possible, so that there is no doubt as to the extent of your collaboration.

I did not collaborate with anyone for this assignment.